# Lab 5. Quantization and Pruning

Sometimes you may want to run your model on edge devices such as Raspberry Pi's, mobile phones or IoT devices. These devices have limited computational resources and memory. Therefore, you need to optimize your model to run on these devices. In this lab, you will learn about some of the techniques that can be used to optimize your model for edge devices.

You will start by training a Keras model. Then you will use this model as a baseline for comparison with the reduced models. You will go through the following techniques of model size reduction (the links contain guides about each technique):

* [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization)
* [quantization aware training](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide)
* [weight pruning](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide)

## Part 0. Preparation

Let's first import a few common libraries that we'll use throughout the lab. Print the version of TensorFlow to check if everything is working correctly.

We'll use the following libraries:

- `tensorflow`: to do the actual ML
- `numpy`: for handling the dataset (MNIST)
- `os`: for reading model files size
- `tempfile`: to create temporary files (for saving gzipped models)
- `zipfile`: to make a gzipped model

In [23]:
import tensorflow as tf
import numpy as np
import os
import tempfile
import zipfile

print(tf.__version__)

2.15.0


Let's also define a few string constants and utility functions to make our code easier to maintain.

In [24]:
# String constants for model filenames
FILE_WEIGHTS = 'baseline_weights.h5'
FILE_NON_QUANTIZED_H5 = 'non_quantized.h5'
FILE_NON_QUANTIZED_TFLITE = 'non_quantized.tflite'
FILE_PT_QUANTIZED = 'post_training_quantized.tflite'
FILE_QAT_QUANTIZED = 'quant_aware_quantized.tflite'
FILE_PRUNED_MODEL_H5 = 'pruned_model.h5'
FILE_PRUNED_QUANTIZED_TFLITE = 'pruned_quantized.tflite'
FILE_PRUNED_NON_QUANTIZED_TFLITE = 'pruned_non_quantized.tflite'

# Dictionaries to hold measurements
MODEL_SIZE = {}
ACCURACY = {}

In [25]:
def print_metric(metric_dict, metric_name):
  '''Prints key and values stored in a dictionary'''
  for metric, value in metric_dict.items():
    print(f'{metric_name} for {metric}: {value}')


def model_builder():
  '''Returns a shallow CNN for training on the MNIST dataset'''

  keras = tf.keras

  # Define the model architecture.
  model = keras.Sequential([
    keras.layers.InputLayer(input_shape=(28, 28)), # Input layer
    keras.layers.Reshape(target_shape=(28, 28, 1)), # Reshape to 28x28x1
    keras.layers.Conv2D(filters=12, kernel_size=(3, 3), activation='relu'), # Input layer
    keras.layers.MaxPooling2D(pool_size=(2, 2)), # Downsizing images
    keras.layers.Flatten(), # Images are 3D, flatten to 1D
    keras.layers.Dense(10, activation='softmax') # Output layer (with 10 possible outputs for each digit)
  ])

  return model


def evaluate_tflite_model(filename, x_test, y_test):
  '''
  Measures the accuracy of a given TF Lite model and test set

  Args:
    filename (string) - filename of the model to load
    x_test (numpy array) - test images
    y_test (numpy array) - test labels

  Returns
    float showing the accuracy against the test set
  '''

  # Initialize the TF Lite Interpreter and allocate tensors
  interpreter = tf.lite.Interpreter(model_path=filename)
  interpreter.allocate_tensors()

  # Get input and output index
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Initialize empty predictions list
  prediction_digits = []

  # Run predictions on every image in the "test" dataset.
  for i, test_image in enumerate(x_test):
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    digit = np.argmax(output()[0])
    prediction_digits.append(digit)

  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_digits = np.array(prediction_digits)
  accuracy = (prediction_digits == y_test).mean()

  return accuracy


def get_gzipped_model_size(file):
  '''Returns size of gzipped model, in bytes.'''
  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)

  return os.path.getsize(zipped_file)

### Download and prepare the MNIST dataset

You will be using the [MNIST](https://keras.io/api/datasets/mnist/) dataset which is hosted in [Keras Datasets](https://keras.io/api/datasets/). The MNIST dataset contains 60,000 training images of handwritten digits from zero to nine and 10,000 images for testing. The images are grayscale and 28 x 28 pixels. You will use this dataset to train a model to classify the images into 10 classes (one for each digit).

In [26]:
# Load MNIST dataset
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# Normalize the input image so that each pixel value is between 0 to 1.
train_images = train_images / 255.0
test_images = test_images / 255.0

## Part 1. Baseline model

Let's first build and train a Keras model. This will be the baseline where you will be comparing the mobile optimized versions later on. This will just be a shallow CNN with a softmax output to classify a given MNIST digit. You can review the `model_builder()` function in the utilities at the top of this notebook but we also printed the model summary below to show the architecture.

You will also save the weights so you can reinitialize the other models later the same way. This is not needed in real projects but for this demo notebook, it would be good to have the same initial state later so you can compare the effects of the optimizations.

In [27]:
# Create the baseline model
baseline_model = model_builder()

# Save the initial weights for use later
baseline_model.save_weights(FILE_WEIGHTS)

# Print the model summary
baseline_model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 12)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2028)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                20290     
                                                                 
Total params: 20410 (79.73 KB)
Trainable params: 20410 (79.73 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

Now, compile and train the model. In practice, it's best to shuffle the train set but for this lab, it is set to `False` for reproducibility of the results. One epoch will reach around 91% accuracy.

In [28]:
# Setup the model for training
baseline_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
baseline_model.fit(train_images, train_labels, epochs=1, shuffle=False)

1875/1875 [==============================] - 35s 18ms/step - loss: 0.2865 - accuracy: 0.9194


Let's save the accuracy of the model against the test set so you can compare later.

In [29]:

# Get the baseline accuracy
_, ACCURACY['baseline Keras model'] = baseline_model.evaluate(test_images, test_labels)

313/313 [==============================] - 2s 5ms/step - loss: 0.1721 - accuracy: 0.9455


Next, you will save the Keras model as a file and record its size as well.

In [30]:
# Save the Keras model
baseline_model.save(FILE_NON_QUANTIZED_H5, include_optimizer=False)

# Save and get the model size
MODEL_SIZE['baseline h5'] = os.path.getsize(FILE_NON_QUANTIZED_H5)

# Print records so far
print_metric(ACCURACY, "test accuracy")
print_metric(MODEL_SIZE, "model size in bytes")

test accuracy for baseline Keras model: 0.9455000162124634
model size in bytes for baseline h5: 98928


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Convert the model to TF Lite format

Next, you will convert the model to [Tensorflow Lite (TF Lite)](https://www.tensorflow.org/lite/guide) format. This is designed to make Tensorflow models more efficient and lightweight when running on mobile, embedded, and IoT devices.

You can convert a Keras model with TF Lite's [Converter](https://www.tensorflow.org/lite/convert/index) class and we've incorporated it in the short helper function below. Notice that there is a `quantize` flag which you can use to quantize the model.

Complete the function below:

1. Create a TFLiteConverter for the given model.
2. Add default optimization when the parameter `quantize` is `True`.
3. Convert the model.

Check the [documentation](https://www.tensorflow.org/lite/performance/post_training_quantization) for more information.

In [31]:
def convert_tflite(model, filename, quantize=False):
  '''
  Converts the model to TF Lite format and writes to a file

  Args:
    model (Keras model) - model to convert to TF Lite
    filename (string) - string to use when saving the file
    quantize (bool) - flag to indicate quantization

  Returns:
    None
  '''

  # Initialize the converter
  converter = tf.lite.TFLiteConverter.from_keras_model(model)

  # Set for quantization if flag is set to True
  if quantize:
    converter.optimizations = [tf.lite.Optimize.DEFAULT]

  # Convert the model
  tflite_model = converter.convert()

  # Save the model.
  with open(filename, 'wb') as f:
    f.write(tflite_model)

You will use the helper function to convert the Keras model then get its size and accuracy. Take note that this is *not yet* quantized.

In [32]:
# Convert baseline model
convert_tflite(baseline_model, FILE_NON_QUANTIZED_TFLITE)

You will notice that there is already a slight decrease in model size when converting to `.tflite` format.

In [33]:
MODEL_SIZE['non quantized tflite'] = os.path.getsize(FILE_NON_QUANTIZED_TFLITE)

print_metric(MODEL_SIZE, 'model size in bytes')

model size in bytes for baseline h5: 98928
model size in bytes for non quantized tflite: 85096


The accuracy will also be nearly identical when converting between formats. You can setup a TF Lite model for input-output using its [Interpreter](https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter) class. This is shown in the `evaluate_tflite_model()` helper function provided in the `Utilities` section earlier.

*Note: If you see a `Runtime Error: There is at least 1 reference to internal data in the interpreter in the form of a numpy array or slice.` , please try re-running the cell.*

In [34]:
ACCURACY['non quantized tflite'] = evaluate_tflite_model(FILE_NON_QUANTIZED_TFLITE, test_images, test_labels)

In [35]:
print_metric(ACCURACY, 'test accuracy')

test accuracy for baseline Keras model: 0.9455000162124634
test accuracy for non quantized tflite: 0.9455


## Part 2. Post-Training Quantization

Now that you have the baseline metrics, you can now observe the effects of quantization. This process involves converting floating point representations into integer to reduce model size and achieve faster computation.

As shown in the `convert_tflite()` helper function earlier, you can easily do [post-training quantization](https://www.tensorflow.org/lite/performance/post_training_quantization) with the TF Lite API. You just need to set the converter optimization and assign an [Optimize](https://www.tensorflow.org/api_docs/python/tf/lite/Optimize) Enum.

You will set the `quantize` flag to do that and get the metrics again.

In [36]:
# Convert and quantize the baseline model
convert_tflite(baseline_model, FILE_PT_QUANTIZED, quantize=True)

In [37]:
# Get the model size
MODEL_SIZE['post training quantized tflite'] = os.path.getsize(FILE_PT_QUANTIZED)

print_metric(MODEL_SIZE, 'model size')

model size for baseline h5: 98928
model size for non quantized tflite: 85096
model size for post training quantized tflite: 24336


You should see around a 4X reduction in model size in the quantized version. This comes from converting the 32 bit representations (float) into 8 bits (integer).



In [38]:
ACCURACY['post training quantized tflite'] = evaluate_tflite_model(FILE_PT_QUANTIZED, test_images, test_labels)

In [39]:

print_metric(ACCURACY, 'test accuracy')

test accuracy for baseline Keras model: 0.9455000162124634
test accuracy for non quantized tflite: 0.9455
test accuracy for post training quantized tflite: 0.9457


You can expect the accuracy to not be the same when quantizing the model. Most of the time it will decrease but in some cases, it can even increase. Again, this can be attributed to the loss of precision when you remove the extra bits from the float data.

## Part 3. Quantization Aware Training

When post-training quantization results in loss of accuracy that is unacceptable for your application, you can consider doing [quantization aware training](https://www.tensorflow.org/model_optimization/guide/quantization/training) before quantizing the model. This simulates the loss of precision by inserting fake quant nodes in the model during training. That way, your model will learn to adapt with the loss of precision to get more accurate predictions.

The [Tensorflow Model Optimization Toolkit](https://www.tensorflow.org/model_optimization) provides a [quantize_model()](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/quantization/keras/quantize_model) method to do this quickly and you will see that below. But first, let's install the toolkit into the notebook environment.

In [40]:
# Install the toolkit
!pip install tensorflow_model_optimization

You will build the baseline model again but this time, pass it into the `quantize_model()` method to indicate quantization aware training (note the `TODO`). Check the [documentation](https://www.tensorflow.org/model_optimization/guide/quantization/training_comprehensive_guide) for more information.

Take note that in case you decide to pass in a model that is already trained, then make sure to recompile before you continue training.

In [41]:
import tensorflow_model_optimization as tfmot

# method to quantize a Keras model
quantize_model = tfmot.quantization.keras.quantize_model

# Define the model architecture.
model_to_quantize = model_builder()

# Reinitialize weights with saved file
model_to_quantize.load_weights(FILE_WEIGHTS)

# Quantize the model
q_aware_model = tfmot.quantization.keras.quantize_model(model_to_quantize)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer_1 (Quantize  (None, 28, 28)            3         
 Layer)                                                          
                                                                 
 quant_reshape_3 (QuantizeW  (None, 28, 28, 1)         1         
 rapperV2)                                                       
                                                                 
 quant_conv2d_3 (QuantizeWr  (None, 26, 26, 12)        147       
 apperV2)                                                        
                                                                 
 quant_max_pooling2d_3 (Qua  (None, 13, 13, 12)        1         
 ntizeWrapperV2)                                                 
                                                                 
 quant_flatten_3 (QuantizeW  (None, 2028)             

You may have noticed a slight difference in the model summary above compared to the baseline model summary in the earlier sections. The total params count increased as expected because of the nodes added by the `quantize_model()` method.

With that, you can now train the model. You will notice that the accuracy is a bit lower because the model is simulating the loss of precision. The training will take a bit longer if you want to achieve the same training accuracy as the earlier run. For this exercise though, we will keep to 1 epoch.

In [42]:
# Train the model
q_aware_model.fit(train_images, train_labels, epochs=1, shuffle=False)

1875/1875 [==============================] - 37s 19ms/step - loss: 0.2886 - accuracy: 0.9189


You can then get the accuracy of the Keras model before and after quantizing the model. The accuracy is expected to be nearly identical because the model is trained to counter the effects of quantization.

In [43]:
# Get the accuracy of the quantization aware trained model (not yet quantized)
_, ACCURACY['quantization aware non-quantized'] = q_aware_model.evaluate(test_images, test_labels, verbose=0)
print_metric(ACCURACY, 'test accuracy')

test accuracy for baseline Keras model: 0.9455000162124634
test accuracy for non quantized tflite: 0.9455
test accuracy for post training quantized tflite: 0.9457
test accuracy for quantization aware non-quantized: 0.947700023651123


In [44]:
# Convert and quantize the model.
convert_tflite(q_aware_model, FILE_QAT_QUANTIZED, quantize=True)

# Get the accuracy of the quantized model
ACCURACY['quantization aware quantized'] = evaluate_tflite_model(FILE_QAT_QUANTIZED, test_images, test_labels)
print_metric(ACCURACY, 'test accuracy')

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:953: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


test accuracy for baseline Keras model: 0.9455000162124634
test accuracy for non quantized tflite: 0.9455
test accuracy for post training quantized tflite: 0.9457
test accuracy for quantization aware non-quantized: 0.947700023651123
test accuracy for quantization aware quantized: 0.9476


## Part 4. Pruning

Let's now move on to another technique for reducing model size: [Pruning](https://www.tensorflow.org/model_optimization/guide/pruning/pruning_with_keras). This process involves zeroing out insignificant (i.e. low magnitude) weights. The intuition is these weights do not contribute as much to making predictions so you can remove them and get the same result. Making the weights sparse helps in compressing the model more efficiently and you will see that in this section.

The Tensorflow Model Optimization Toolkit again has a convenience method for this. The [prune_low_magnitude()](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/prune_low_magnitude) method puts wrappers in a Keras model so it can be pruned during training. You will pass in the baseline model that you already trained earlier. You will notice that the model summary show increased params because of the wrapper layers added by the pruning method.

You can set how the pruning is done during training. Below, you will use [PolynomialDecay](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/PolynomialDecay) to indicate how the sparsity ramps up with each step. Another option available in the library is [Constant Sparsity](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/ConstantSparsity). Use the following parameters for `PolynomialDecay`:

- initial sparsity: 0.50
- final sparsity: 0.80
- begin step: 0
- end step: use `end_step` variable calculated in this cell

Check the [documentation](https://www.tensorflow.org/model_optimization/guide/pruning/comprehensive_guide) for more information.

In [54]:
# Get the pruning method
prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

# Compute end step to finish pruning after 2 epochs.
batch_size = 128
epochs = 2
validation_split = 0.1 # 10% of training set will be used for validation set.

num_images = train_images.shape[0] * (1 - validation_split)
end_step = np.ceil(num_images / batch_size).astype(np.int32) * epochs

# Define pruning schedule.
pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=0.5, final_sparsity=0.8, begin_step=0, end_step=end_step),
}

# Pass in the trained baseline model
model_for_pruning = prune_low_magnitude(baseline_model, **pruning_params)

# `prune_low_magnitude` requires a recompile.
model_for_pruning.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model_for_pruning.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 prune_low_magnitude_reshap  (None, 28, 28, 1)         1         
 e_2 (PruneLowMagnitude)                                         
                                                                 
 prune_low_magnitude_conv2d  (None, 26, 26, 12)        230       
 _2 (PruneLowMagnitude)                                          
                                                                 
 prune_low_magnitude_max_po  (None, 13, 13, 12)        1         
 oling2d_2 (PruneLowMagnitu                                      
 de)                                                             
                                                                 
 prune_low_magnitude_flatte  (None, 2028)              1         
 n_2 (PruneLowMagnitude)                                         
                                                      

You can also peek at the weights of one of the layers in your model. After pruning, you will notice that many of these will be zeroed out.

In [58]:
# Preview model weights
model_for_pruning.weights[1]

<tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 1, 12) dtype=float32, numpy=
array([[[[-0.05545352, -0.02751824, -0.43092608, -0.23403364,
          -0.46647194,  0.3134932 , -0.07886636,  0.29909644,
          -0.13631496, -0.29054266, -0.50881326,  0.38384232]],

        [[ 0.26275235,  0.08423154, -0.15594299,  0.0725574 ,
          -0.01883893, -0.18907684,  0.24468188,  0.17960416,
           0.10258649,  0.24987297, -0.79322726,  0.23318978]],

        [[ 0.10577784,  0.21192959,  0.07148413,  0.21231872,
           0.59356785, -0.29478782,  0.1751113 ,  0.15296899,
           0.11595   ,  0.19913168, -0.8043021 , -0.7168729 ]]],


       [[[ 0.0041661 ,  0.27156353, -0.34725317,  0.10498732,
          -0.7358056 ,  0.12207078,  0.29608732, -0.05619534,
           0.09562663, -0.00385157,  0.3923791 ,  0.5970079 ]],

        [[ 0.30526608,  0.07136765, -0.16917239,  0.2716618 ,
           0.01194071,  0.28488582,  0.0194982 ,  0.1762014 ,
           0.33256507,  0.2976723 ,  0.2657

With that, you can now start re-training the model. Add the [UpdatePruningStep()](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/UpdatePruningStep) callback to the `callbacks` array.

In [59]:
# Callback to update pruning wrappers at each step
callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep()
]

# Train and prune the model
model_for_pruning.fit(train_images, train_labels,
                  epochs=epochs, validation_split=validation_split,
                  callbacks=callbacks)

Epoch 1/2
1688/1688 [==============================] - 23s 12ms/step - loss: 0.1644 - accuracy: 0.9551 - val_loss: 0.1089 - val_accuracy: 0.9720
Epoch 2/2
1688/1688 [==============================] - 19s 11ms/step - loss: 0.1277 - accuracy: 0.9624 - val_loss: 0.0957 - val_accuracy: 0.9758


Now see how the weights in the same layer looks like after pruning.

In [60]:
# Preview model weights
model_for_pruning.weights[1]

<tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 1, 12) dtype=float32, numpy=
array([[[[-0.        ,  0.        , -0.        , -0.        ,
          -0.9055014 ,  0.        ,  0.        ,  0.8037277 ,
          -0.        , -0.        , -0.706316  ,  0.7153444 ]],

        [[-0.        ,  0.        , -0.        , -0.        ,
          -0.        ,  0.        ,  0.        ,  0.        ,
           0.        , -0.        , -1.025887  , -0.        ]],

        [[ 0.        ,  0.        , -0.        , -0.        ,
           0.8345351 ,  0.        ,  0.        ,  0.        ,
           0.        , -0.        , -1.0095204 , -1.0432383 ]]],


       [[[-0.        ,  0.        , -0.        , -0.        ,
          -1.1445985 ,  0.        , -0.        ,  0.        ,
          -0.        , -0.        ,  0.        ,  1.0123976 ]],

        [[ 0.8282131 ,  0.        , -0.        , -0.        ,
          -0.        ,  0.        , -0.        ,  0.        ,
           0.7866553 , -0.        ,  0.    

After pruning, you can remove the wrapper layers to have the same layers and params as the baseline model. You can do that with the [strip_pruning()](https://www.tensorflow.org/model_optimization/api_docs/python/tfmot/sparsity/keras/strip_pruning) method as shown below. You will do this so you can save the model and also export to TF Lite format just like in the previous sections.

In [61]:
# Remove pruning wrappers
model_for_export = tfmot.sparsity.keras.strip_pruning(model_for_pruning)
model_for_export.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 reshape_2 (Reshape)         (None, 28, 28, 1)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 12)        120       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 12)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 2028)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                20290     
                                                                 
Total params: 20410 (79.73 KB)
Trainable params: 20410 (79.73 KB)
Non-trainable params: 0 (0.00 Byte)
__________________

You will see the same model weights but the index is different because the wrappers were removed.

In [62]:
# Preview model weights (index 1 earlier is now 0 because pruning wrappers were removed)
model_for_export.weights[0]

<tf.Variable 'conv2d_2/kernel:0' shape=(3, 3, 1, 12) dtype=float32, numpy=
array([[[[-0.        ,  0.        , -0.        , -0.        ,
          -0.9055014 ,  0.        ,  0.        ,  0.8037277 ,
          -0.        , -0.        , -0.706316  ,  0.7153444 ]],

        [[-0.        ,  0.        , -0.        , -0.        ,
          -0.        ,  0.        ,  0.        ,  0.        ,
           0.        , -0.        , -1.025887  , -0.        ]],

        [[ 0.        ,  0.        , -0.        , -0.        ,
           0.8345351 ,  0.        ,  0.        ,  0.        ,
           0.        , -0.        , -1.0095204 , -1.0432383 ]]],


       [[[-0.        ,  0.        , -0.        , -0.        ,
          -1.1445985 ,  0.        , -0.        ,  0.        ,
          -0.        , -0.        ,  0.        ,  1.0123976 ]],

        [[ 0.8282131 ,  0.        , -0.        , -0.        ,
          -0.        ,  0.        , -0.        ,  0.        ,
           0.7866553 , -0.        ,  0.    

You will notice below that the pruned model will have the same file size as the baseline_model when saved as H5. This is to be expected. The improvement will be noticeable when you compress the model as will be shown in the cell after this.

In [63]:
# Save Keras model
model_for_export.save(FILE_PRUNED_MODEL_H5, include_optimizer=False)

# Get uncompressed model size of baseline and pruned models
MODEL_SIZE['baseline h5'] = os.path.getsize(FILE_NON_QUANTIZED_H5)
MODEL_SIZE['pruned non quantized h5'] = os.path.getsize(FILE_PRUNED_MODEL_H5)

print_metric(MODEL_SIZE, 'model_size in bytes')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


model_size in bytes for baseline h5: 98928
model_size in bytes for non quantized tflite: 85096
model_size in bytes for post training quantized tflite: 24336
model_size in bytes for pruned non quantized h5: 98928


You will use the `get_gzipped_model_size()` helper function in the `Utilities` to compress the models and get its resulting file size. You will notice that the pruned model is about 3 times smaller. This is because of the sparse weights generated by the pruning process. The zeros can be compressed much more efficiently than the low magnitude weights before pruning.

In [64]:
# Get compressed size of baseline and pruned models
MODEL_SIZE['baseline h5 (gzipped)'] = get_gzipped_model_size(FILE_NON_QUANTIZED_H5)
MODEL_SIZE['pruned non quantized h5 (gzipped)'] = get_gzipped_model_size(FILE_PRUNED_MODEL_H5)

print_metric(MODEL_SIZE, "gzipped model size in bytes")

gzipped model size in bytes for baseline h5: 98928
gzipped model size in bytes for non quantized tflite: 85096
gzipped model size in bytes for post training quantized tflite: 24336
gzipped model size in bytes for pruned non quantized h5: 98928
gzipped model size in bytes for baseline h5 (gzipped): 78073
gzipped model size in bytes for pruned non quantized h5 (gzipped): 25659


You can make the model even more lightweight by quantizing the pruned model. This achieves around 10X reduction in compressed model size as compared to the baseline.

In [65]:
# Convert and quantize the pruned model.
pruned_quantized_tflite = convert_tflite(model_for_export, FILE_PRUNED_QUANTIZED_TFLITE, quantize=True)

# Compress and get the model size
MODEL_SIZE['pruned quantized tflite (gzipped)'] = get_gzipped_model_size(FILE_PRUNED_QUANTIZED_TFLITE)
print_metric(MODEL_SIZE, "gzipped model size in bytes")

gzipped model size in bytes for baseline h5: 98928
gzipped model size in bytes for non quantized tflite: 85096
gzipped model size in bytes for post training quantized tflite: 24336
gzipped model size in bytes for pruned non quantized h5: 98928
gzipped model size in bytes for baseline h5 (gzipped): 78073
gzipped model size in bytes for pruned non quantized h5 (gzipped): 25659
gzipped model size in bytes for pruned quantized tflite (gzipped): 8370


As expected, the TF Lite model's accuracy will also be close to the Keras model.

In [66]:
# Get accuracy of pruned Keras and TF Lite models
_, ACCURACY['pruned model h5'] = model_for_pruning.evaluate(test_images, test_labels)
ACCURACY['pruned and quantized tflite'] = evaluate_tflite_model(FILE_PRUNED_QUANTIZED_TFLITE, test_images, test_labels)

print_metric(ACCURACY, 'accuracy')

313/313 [==============================] - 2s 5ms/step - loss: 0.1131 - accuracy: 0.9660
accuracy for baseline Keras model: 0.9455000162124634
accuracy for non quantized tflite: 0.9455
accuracy for post training quantized tflite: 0.9457
accuracy for quantization aware non-quantized: 0.947700023651123
accuracy for quantization aware quantized: 0.9476
accuracy for pruned model h5: 0.9660000205039978
accuracy for pruned and quantized tflite: 0.9659


## Possible extensions

- Improve the model (e.g. <https://colab.research.google.com/drive/1W8fCTLZAO9YKcNRQzYfFf__9CROQLOcZ>) and compare the model sizes after improvement.
- Create a quantized and/or pruned model for a model of your choice or a personal project

